In [14]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import os

In [15]:
spark = SparkSession.builder \
                    .appName('tennis') \
                    .getOrCreate()

In [65]:
folder = "datasets/"
file_columns = []
files65 = []
files49= []

In [66]:
for filename in os.listdir(folder):

    file_path = os.path.join(folder, filename)
    
    df = spark.read.option("header", "true") \
                   .option("inferSchema", "true") \
                   .csv(file_path)
    
    #file_columns.append({filename:df.columns})
    file_columns.append(df.columns)
    if len(df.columns) == 65:
        files65.append(filename)
    else:
        files49.append(filename)

In [67]:
# Removing duplicates
file_columns = list(set(map(tuple, file_columns)))
file_columns = [list(item) for item in file_columns]
file_columns

[['tourney_id',
  'tourney_name',
  'surface',
  'draw_size',
  'tourney_level',
  'tourney_date',
  'match_num',
  'winner1_id',
  'winner2_id',
  'winner_seed',
  'winner_entry',
  'loser1_id',
  'loser2_id',
  'loser_seed',
  'loser_entry',
  'score',
  'best_of',
  'round',
  'winner1_name',
  'winner1_hand',
  'winner1_ht',
  'winner1_ioc',
  'winner1_age',
  'winner2_name',
  'winner2_hand',
  'winner2_ht',
  'winner2_ioc',
  'winner2_age',
  'loser1_name',
  'loser1_hand',
  'loser1_ht',
  'loser1_ioc',
  'loser1_age',
  'loser2_name',
  'loser2_hand',
  'loser2_ht',
  'loser2_ioc',
  'loser2_age',
  'winner1_rank',
  'winner1_rank_points',
  'winner2_rank',
  'winner2_rank_points',
  'loser1_rank',
  'loser1_rank_points',
  'loser2_rank',
  'loser2_rank_points',
  'minutes',
  'w_ace',
  'w_df',
  'w_svpt',
  'w_1stIn',
  'w_1stWon',
  'w_2ndWon',
  'w_SvGms',
  'w_bpSaved',
  'w_bpFaced',
  'l_ace',
  'l_df',
  'l_svpt',
  'l_1stIn',
  'l_1stWon',
  'l_2ndWon',
  'l_SvGms',
  

In [68]:
print(f"Number of different column sets: {len(file_columns)}")
print(f"Number of columns in the first set: {len(file_columns[0])}")
print(f"Number of columns in the second set: {len(file_columns[1])}")

Number of different column sets: 2
Number of columns in the first set: 65
Number of columns in the second set: 49


In [73]:
col1 = set(file_columns[0])
col2 = set(file_columns[1])
diff1 = col1 - col2 # Columns in Doubles files that are not in Singles files
diff2 = col2 - col1 # Columns in Singles files that are not in Doubles files
# ^ to get both

In [70]:
diff1

{'loser1_age',
 'loser1_hand',
 'loser1_ht',
 'loser1_id',
 'loser1_ioc',
 'loser1_name',
 'loser1_rank',
 'loser1_rank_points',
 'loser2_age',
 'loser2_hand',
 'loser2_ht',
 'loser2_id',
 'loser2_ioc',
 'loser2_name',
 'loser2_rank',
 'loser2_rank_points',
 'winner1_age',
 'winner1_hand',
 'winner1_ht',
 'winner1_id',
 'winner1_ioc',
 'winner1_name',
 'winner1_rank',
 'winner1_rank_points',
 'winner2_age',
 'winner2_hand',
 'winner2_ht',
 'winner2_id',
 'winner2_ioc',
 'winner2_name',
 'winner2_rank',
 'winner2_rank_points'}

In [71]:
diff2

{'loser_age',
 'loser_hand',
 'loser_ht',
 'loser_id',
 'loser_ioc',
 'loser_name',
 'loser_rank',
 'loser_rank_points',
 'winner_age',
 'winner_hand',
 'winner_ht',
 'winner_id',
 'winner_ioc',
 'winner_name',
 'winner_rank',
 'winner_rank_points'}

### Some should be added, others should be just changed (e.g. loser_age -> loser1_age)

In [74]:
# Files with 65 columns
files65

['atp_matches_doubles_2007.csv',
 'atp_matches_doubles_2019.csv',
 'atp_matches_doubles_2012.csv',
 'atp_matches_doubles_2018.csv',
 'atp_matches_doubles_2020.csv',
 'atp_matches_doubles_2011.csv',
 'atp_matches_doubles_2016.csv',
 'atp_matches_doubles_2000.csv',
 'atp_matches_doubles_2017.csv',
 'atp_matches_doubles_2008.csv',
 'atp_matches_doubles_2004.csv',
 'atp_matches_doubles_2013.csv',
 'atp_matches_doubles_2009.csv',
 'atp_matches_doubles_2001.csv',
 'atp_matches_doubles_2006.csv',
 'atp_matches_doubles_2014.csv',
 'atp_matches_doubles_2003.csv',
 'atp_matches_doubles_2015.csv',
 'atp_matches_doubles_2010.csv',
 'atp_matches_doubles_2005.csv',
 'atp_matches_doubles_2002.csv']

In [75]:
# Files with 49 columns
files49

['atp_matches_1984.csv',
 'atp_matches_qual_chall_1980.csv',
 'atp_matches_2009.csv',
 'atp_matches_futures_2018.csv',
 'atp_matches_2022.csv',
 'atp_matches_futures_2016.csv',
 'atp_matches_1990.csv',
 'atp_matches_2016.csv',
 'atp_matches_futures_1995.csv',
 'atp_matches_qual_chall_2003.csv',
 'atp_matches_qual_chall_2019.csv',
 'atp_matches_qual_chall_2000.csv',
 'atp_matches_2017.csv',
 'atp_matches_qual_chall_2012.csv',
 'atp_matches_2001.csv',
 'atp_matches_2007.csv',
 'atp_matches_1985.csv',
 'atp_matches_futures_2006.csv',
 'atp_matches_futures_2009.csv',
 'atp_matches_qual_chall_1999.csv',
 'atp_matches_1971.csv',
 'atp_matches_2013.csv',
 'atp_matches_2021.csv',
 'atp_matches_amateur.csv',
 'atp_matches_futures_2017.csv',
 'atp_matches_futures_2015.csv',
 'atp_matches_1991.csv',
 'atp_matches_qual_chall_2020.csv',
 'atp_matches_1992.csv',
 'atp_matches_futures_2013.csv',
 'atp_matches_qual_chall_2021.csv',
 'atp_matches_1979.csv',
 'atp_matches_2003.csv',
 'atp_matches_qual_c